# **MixSync: Song Key & BPM Finder**
With this application, you can accurately find and sort songs by key and BPM,
curated with meticulous verification, and filter them by various attributes for a seamless music-matching experience.

In [ ]:
from google.colab import drive
drive.mount ('/gdrive')

In [ ]:
# @title For Western Songs
import pandas as pd
import time

# Delay function for animation purposes
def delay():
    time.sleep(0.5)


# Check if input is numeric
def is_numeric(input_str):
    try:
        float(input_str)
        return True
    except ValueError:
        return False


# Sort database by specified column
def sorting(sort_column, sort_order):
    if sort_order.lower() == "Ascending":
        sorted_df = database.sort_values(by=sort_column)
    elif sort_order.lower() == "Descending":
        sorted_df = database.sort_values(by=sort_column, ascending=False)
    sorted_df.to_csv(db_name, index=False)
    return sorted_df


# Find songs within specified BPM range
def bpm_finder(database, bpm, bpm_thresh):
    delay()
    print(f"Finding songs that are ±{bpm_thresh} BPM around {bpm} BPM...")
    bpm_min = bpm - bpm_thresh
    bpm_max = bpm + bpm_thresh
    find_bpm = database[(database['BPM'].between(bpm_min, bpm_max))]
    return find_bpm


# Find songs within specified key range
def key_finder(find_bpm, key, key_thresh):
    delay()
    print(f"Finding songs that are {key_thresh} keys apart from {key}...")
    key_index = key_list.index(key)
    surrounding_keys = []
    for counter in range(-key_thresh, key_thresh + 1):
        if 0 <= key_index + counter < len(key_list):
            surrounding_keys.append(key_list[key_index + counter])
    find_keybpm = find_bpm[find_bpm['KEY'].isin(surrounding_keys)]
    return find_keybpm


# MAIN PROGRAM
if __name__ == "__main__":
  div = "=" * 15

  # List of keys
  key_list = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]

  # Read the database based on db_name
  db_name = 'westerndb.csv'

  # @markdown Folder Location
  location = "/gdrive/My Drive/MixSync" # @param {type:"string"}
  location = location + "/westerndb.csv"

  database = pd.read_csv(location)
  columns_to_display = ['ARTIST', 'TITLE', 'KEY', 'BPM']
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)

  # User input: BPM, Key, BPM and Key Threshold
  while True:
      try:
          # @markdown BPM: Beats per Minute
          bpm = 70 # @param {type:"slider", min:70, max:139, step:1}
          if bpm not in range(70, 140):
              raise ValueError("BPM must be between 70 and 139.")
          break
      except ValueError:
          print("Input is not a valid number or outside the range.")

  while True:
      # @markdown Key: The pitch around which the song revolves melodically
      key = "C" # @param ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
      if key in key_list:
          break
      else:
          print("Input is not a valid key.")

  while True:
      try:
          # @markdown BPM Threshold:
          # @markdown The allowable deviation from the specified BPM to include other songs that have a BPM close to the specified value.
          bpm_thresh = 5 # @param {type:"slider", min:0, max:35, step:1}
          break
      except ValueError:
          print("Input is not a valid number.")

  while True:
      try:
          # @markdown Key Threshold:
          # @markdown The number of musical keys away from the specified key that you are willing to consider when searching for songs.
          key_thresh = 0 # @param {type:"slider", min:0, max:6, step:1}
          break
      except ValueError:
          print("Input is not a valid number.")

  # Adjust bpm and bpm_thresh if bpm is not in the range
  if bpm not in range(70, 140):
      bpm /= 2
      bpm_thresh /= 2

  # User input: Ask if the list needs to be sorted
  # @markdown Sort the list? or not?
  sort = "NO SORT" # @param ["NO SORT", "ARTIST", "TITLE", "KEY", "BPM"]
  sort_order = "Ascending" # @param ["Ascending", "Descending"]

  if sort == "NO SORT":
      sort_db = database  # Use the unsorted database
  else:
      sort_db = sorting(sort, sort_order)

  # Function calls: Find songs
  find_bpm = bpm_finder(sort_db, bpm, bpm_thresh)
  find_keybpm = key_finder(find_bpm, key, key_thresh)

  # Display results
  delay()
  print(f"{div} SONGS {div}")
  print(find_keybpm[columns_to_display])
  print(f"{div}======={div}")

In [ ]:
# @title For Anime Songs
import pandas as pd
import time

# Delay function for animation purposes
def delay():
    time.sleep(0.5)


# Check if input is numeric
def is_numeric(input_str):
    try:
        float(input_str)
        return True
    except ValueError:
        return False


# Sort database by specified column
def sorting(sort_column, sort_order):
    if sort_order.lower() == "Ascending":
        sorted_df = database.sort_values(by=sort_column)
    elif sort_order.lower() == "Descending":
        sorted_df = database.sort_values(by=sort_column, ascending=False)
    sorted_df.to_csv(db_name, index=False)
    return sorted_df


# Find songs within specified BPM range
def bpm_finder(database, bpm, bpm_thresh):
    delay()
    print(f"Finding songs that are ±{bpm_thresh} BPM around {bpm} BPM...")
    bpm_min = bpm - bpm_thresh
    bpm_max = bpm + bpm_thresh
    find_bpm = database[(database['BPM'].between(bpm_min, bpm_max))]
    return find_bpm


# Find songs within specified key range
def key_finder(find_bpm, key, key_thresh):
    delay()
    print(f"Finding songs that are {key_thresh} keys apart from {key}...")
    key_index = key_list.index(key)
    surrounding_keys = []
    for counter in range(-key_thresh, key_thresh + 1):
        if 0 <= key_index + counter < len(key_list):
            surrounding_keys.append(key_list[key_index + counter])
    find_keybpm = find_bpm[find_bpm['KEY'].isin(surrounding_keys)]
    return find_keybpm


# MAIN PROGRAM
if __name__ == "__main__":
  div = "=" * 15

  # List of keys
  key_list = ["C Major", "C# Major", "D Major", "D# Major", "E Major", "F Major",
              "F# Major", "G Major", "G# Major", "A Major", "A# Major", "B Major"]

  # Read the database based on db_name
  db_name = 'keybpm.csv'

  # @markdown Folder Location
  location = "/gdrive/My Drive/MixSync" # @param {type:"string"}
  location = location + "/keybpm.csv"

  database = pd.read_csv(location)
  columns_to_display = ['BPM', 'TITLE', 'KEY', 'YEAR']
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)

  # User input: BPM, Key, BPM and Key Threshold
  while True:
      try:
          # @markdown BPM: Beats per Minute
          bpm = 70 # @param {type:"slider", min:70, max:139, step:1}
          if bpm not in range(70, 140):
              raise ValueError("BPM must be between 70 and 139.")
          break
      except ValueError:
          print("Input is not a valid number or outside the range.")

  while True:
      # @markdown Key: The pitch around which the song revolves melodically
      key = "C Major" # @param ["C Major", "C# Major", "D Major", "D# Major", "E Major", "F Major", "F# Major", "G Major", "G# Major", "A Major", "A# Major", "B Major"]
      if key in key_list:
          break
      else:
          print("Input is not a valid key.")

  while True:
      try:
          # @markdown BPM Threshold:
          # @markdown The allowable deviation from the specified BPM to include other songs that have a BPM close to the specified value.
          bpm_thresh = 3 # @param {type:"slider", min:0, max:35, step:1}
          break
      except ValueError:
          print("Input is not a valid number.")

  while True:
      try:
          # @markdown Key Threshold:
          # @markdown The number of musical keys away from the specified key that you are willing to consider when searching for songs.
          key_thresh = 0 # @param {type:"slider", min:0, max:6, step:1}
          break
      except ValueError:
          print("Input is not a valid number.")

  # Adjust bpm and bpm_thresh if bpm is not in the range
  if bpm not in range(70, 140):
      bpm /= 2
      bpm_thresh /= 2

  # User input: Ask if the list needs to be sorted
  # @markdown Sort the list? or not?
  sort = "NO SORT" # @param ["NO SORT", "ARTIST", "TITLE", "KEY", "BPM"]
  sort_order = "Ascending" # @param ["Ascending", "Descending"]

  if sort == "NO SORT":
      sort_db = database  # Use the unsorted database
  else:
      sort_db = sorting(sort, sort_order)

  # Function calls: Find songs
  find_bpm = bpm_finder(sort_db, bpm, bpm_thresh)
  find_keybpm = key_finder(find_bpm, key, key_thresh)

  # Display results
  delay()
  print(f"{div} SONGS {div}")
  print(find_keybpm[columns_to_display])
  print(f"{div}======={div}")